<img src="https://pages.cnpem.br/workshopbioimagens/wp-content/uploads/sites/166/2023/06/logo-ilum-2048x382.png" alt="Descrição da imagem" style="width: 1000px; height: auto; ">
 
<div style=" padding: 10px; font-size:25px; text-align: center;">
<strong> 👹 Fera Formidável 4.12👹 </strong> 
</div>
 
<div style=" padding: 10px; font-size: 50px; text-align: center;">
<strong> Oto come mocotÓ </strong> 
</div>
<div class="alert alert-warning">
<div style="text-align: center; font-size: 15px"><b>Objetivo:</b>  Encontre pelo menos 10 palíndromos diferentes de 5 letras. Estes palíndromos devem ter pelo menos uma vogal. Não é necessário que eles formem palavras válidas em português ou qualquer outro idioma.</div>
</div>
 
<div style=" padding: 10px; font-size: 15px; text-align: center;">
<strong>Júlia Guedes Almeida dos Santos & Yasmin Barbosa Shimizu</strong></div>
<div style=" padding: 10px; font-size: 15px; text-align: center;">
Prof. Dr. Daniel R. Cassar</div>

## 📝 **Introdução**  

> "Socorram-me, subi no ônibus em Marrocos!"

<p align="justify">
A frase acima é um dos exemplos mais conhecidos de palíndromo. Esse conceito, segundo o Dicionário Online de Lingua Portuguesa, pode ser definido como "Uma palavra ou frase que pode ser lida no seu sentido normal, da esquerda para a direita, bem como no sentido contrário, da direita para a esquerda, sem que haja mudança nas palavras que a formam e no seu significado" [1].
</p>

<p align="justify">
No contexto de algoritmos genéticos, seguindo uma lógica semelhante ao problema da senha, no qual palavras/frases são criadas a partir de uma função de criação de população, palíndromos podem ser descobertos a partir dessa construção. No notebook em questão, 10 palíndromos, formados por 5 letras cada, serão encontrados. A única restrição é que todos precisam ter pelo menos uma vogal. Para que isso aconteça, a função objetiva atribuirá uma penalização proporcional a maior distância entre uma letra da palavra e uma vogal. Para o fitness total do palíndromo, será considerada a distância entre o primeiro e o último elemento somada a distância entre o segundo e o penúltimo, somada a essa penalização. Ademais, todos os indivíduos criados inicialmente pela função de criação da população precisarão ser compostos por uma vogal.
</p> 

<p align="justify">
Além dos operadores de seleção (seleção por torneio), cruzamento (cruzamento de ponto duplo) e mutação (mutação de salto), será implementada uma nova função de mutação. Essa, caso a função não tenha nenhuma letra repetida, irá sortear um determinado gene e mudará seu valor para um igual a outro gene sorteado. Dessa forma, o conceito de letras repetidas na palavra será introduzido, o que pode auxiliar na convergência.
</p>

## 📚 **Importação de bibliotecas & funções** 

Em primeiro lugar, precisamos importar as bibliotecas necessárias para resolução do problema. A biblioteca ``string`` será utilizada para a definição dos caracteres possíveis para o palíndromo (todas as letras minúsculas do alfabeto). As demais funções serão importadas do script feito para a resolução desse trabalho.

In [1]:
from string import ascii_lowercase

from funcoes_palindromos import populacao_palindromo as cria_populacao
from funcoes_palindromos import funcao_objetivo_pop_palindromo as funcao_objetivo
from funcoes_palindromos import selecao_torneio_min as funcao_selecao
from funcoes_palindromos import cruzamento_ponto_duplo as funcao_cruzamento
from funcoes_palindromos import mutacao_salto as funcao_mutacao1
from funcoes_palindromos import mutacao_letra_repetida as funcao_mutacao2

## 🔠 **Definição dos parâmetros** 

Com as funções definidas, a próxima etapa consiste na definição dos parâmetros para a implementação do algoritmo genético. Em primeiro lugar, os caracteres possíveis, assim como mencionado na seção de importações, consistirá em todas as letras minúsculas do alfabeto.

In [2]:
CARACTERES_POSSIVEIS = ascii_lowercase

Em relação aos parâmetros, temos que esses serão definidos como:

* **Tamanho da população**: Cada geração será composta por apenas 10 indivíduos (espaço de busca reduzido em relação a problemas como o da senha, que podem abranger letras maiúsculas e digítos);

* **Taxa de cruzamento**: Definida como 0.7 (ou 70%), define a probabilidade de dois indivíduos — "pai" e "mãe" — serem cruzados para gerar novos indivíduos;

* **Taxa de mutação**: Estabelecida em 5%, considerando que duas funções de mutação serão estabelecidas;

* **Tamanho do torneio**: Cada seleção por torneio considerará 3 indivíduos sorteados a partir de uma amostragem da população, dentre os quais apenas o melhor será escolhido;

In [3]:
TAMANHO_POPULACAO = 10
CHANCE_DE_CRUZAMENTO = 0.7
CHANCE_DE_MUTACAO = 0.05
TAMANHO_TORNEIO = 3

Para obter 10 palíndromos, os resultados gerados pelo algoritmo genético serão armazenados em uma lista. Enquanto a lista não contiver 10 indivíduos válidos, um loop while reiniciará as gerações do algoritmo, executando-o novamente. Ao final de cada execução, os melhores indivíduos serão exibidos junto com a geração em que foram encontrados e abaixo de identificador correspondente ao número do palíndromo gerado. Como operadores, serão utilizados:


* **Seleção por torneio**: Um conjunto de $n$ indivíduos (definido pela variável TAMANHO_TORNEIO) é selecionado aleatoriamente. Dentre eles, os $k$ mais aptos — isto é, aqueles com menor valor de função de fitness, indicando maior proximidade de formar um palíndromo — são escolhidos para compor a próxima geração.

* **Cruzamento de ponto duplo**: Dois pontos ao longo do genótipo dos pais são selecionados aleatoriamente, e os segmentos entre esses pontos são trocados entre os indivíduos, gerando dois descendentes com características combinadas.

* **Mutação por salto**: Um gene aleatório do indivíduo é substituído por uma letra situada $n$ posições antes ou depois na ordem alfabética, promovendo diversidade genética.

Na função responsável por criar os candidatos iniciais, cada indivíduo é composto por 5 letras aleatórias, com a restrição de que pelo menos uma delas seja uma vogal. Para garantir isso, o processo consiste em sortear 4 letras quaisquer e, em seguida, inserir uma vogal aleatória em uma posição aleatória do genótipo.

Como adaptações específicas para o problema de geração de palíndromos, foram desenvolvidas:

* **Função objetivo**: Avalia a qualidade de um indivíduo com base na soma das distâncias entre o primeiro e o último caracteres, e entre o segundo e o penúltimo. Como penalização adicional, caso o indivíduo não contenha nenhuma vogal, é aplicada uma penalidade proporcional — 10 vezes a maior distância entre uma letra da palavra e uma vogal (calculada a partir do produto cartesiano entre as vogais existentes e as letras da palavra).

* **Função de mutação por letra repetida**: Se um indivíduo for sorteado para mutação e não possuir letras repetidas, um de seus genes será alterado para copiar o valor de outro gene aleatório do mesmo indivíduo. Essa abordagem favorece a formação de estruturas repetidas, essenciais para a construção de palíndromos, especialmente nas primeiras gerações.



Com isso estabelecido, é possível iniciar o loop de treinamento do algoritmo genético.

In [4]:
novos_palindromos = []

p = 1
while len(novos_palindromos) < 10:
    print(f"PALÍNDROMO {p}")
    populacao = cria_populacao(TAMANHO_POPULACAO, CARACTERES_POSSIVEIS)
    menor_fitness_geral = float("inf")  
    geracao = 0

    while menor_fitness_geral > 0:
        # Seleção
        fitness = funcao_objetivo(populacao)        
        selecionados = funcao_selecao(populacao, fitness, TAMANHO_TORNEIO)
        
        # Cruzamento
        proxima_geracao = []
        for pai, mae in zip(selecionados[::2], selecionados[1::2]):
            individuo1, individuo2 = funcao_cruzamento(pai, mae, CHANCE_DE_CRUZAMENTO)
            proxima_geracao.append(individuo1)
            proxima_geracao.append(individuo2)
        
        # Mutação
        funcao_mutacao1(proxima_geracao, CHANCE_DE_MUTACAO, CARACTERES_POSSIVEIS)
        funcao_mutacao2(proxima_geracao, CHANCE_DE_MUTACAO)
        
        # Nova geração
        populacao = proxima_geracao
        geracao += 1
        
        # Verifica melhor indivíduo
        fitness = funcao_objetivo(populacao)
        menor_fitness_observado = min(fitness)
        if menor_fitness_observado < menor_fitness_geral:
            menor_fitness_geral = menor_fitness_observado
            indice = fitness.index(menor_fitness_observado)
            candidato = populacao[indice]
            print(f'Geração {geracao}: {"".join(candidato)}')
    p+=1
    novos_palindromos.append(candidato)

PALÍNDROMO 1
Geração 1: ipcoc
Geração 6: iococ
Geração 10: iocod
Geração 20: hocod
Geração 27: gocod
Geração 34: gocoe
Geração 49: focoe
Geração 51: focof
PALÍNDROMO 2
Geração 1: eiumi
Geração 2: ejkmi
Geração 3: fjkmi
Geração 9: jjkmi
Geração 13: ijkmi
Geração 37: ikkmi
Geração 60: ilkmi
Geração 63: imkmi
PALÍNDROMO 3
Geração 1: kfmai
Geração 9: kfmbi
Geração 12: jfmbi
Geração 20: ifmbi
Geração 31: ifmci
Geração 39: ifldi
Geração 57: ieldi
Geração 61: idldi
PALÍNDROMO 4
Geração 1: llyel
Geração 5: lkuel
Geração 24: ljuel
Geração 29: liuel
Geração 30: lhuel
Geração 37: lhufl
Geração 54: lhugl
Geração 57: lgugl
PALÍNDROMO 5
Geração 1: uysxy
Geração 7: uysyy
Geração 42: uysyx
Geração 73: uyryw
Geração 148: uyryv
Geração 152: uyryu
PALÍNDROMO 6
Geração 1: yokmu
Geração 2: mokmm
Geração 8: moknm
Geração 11: mokom
PALÍNDROMO 7
Geração 1: imonk
Geração 2: imnnj
Geração 11: innnj
Geração 42: innni
PALÍNDROMO 8
Geração 1: ihjeg
Geração 4: ihjhg
Geração 27: ihjhh
Geração 37: ihjhi
PALÍNDROMO 9


Como foi possível observar, grande parte dos palíndromos foram gerados em menos de 20 gerações, o que demonstra a boa performance do algoritmo genético implementado. Abaixo, é possível observar, em forma de lista, os palíndromos gerados pelo algoritmo:

In [6]:
novos_palindromos

[['f', 'o', 'c', 'o', 'f'],
 ['i', 'm', 'k', 'm', 'i'],
 ['i', 'd', 'l', 'd', 'i'],
 ['l', 'g', 'u', 'g', 'l'],
 ['u', 'y', 'r', 'y', 'u'],
 ['m', 'o', 'k', 'o', 'm'],
 ['i', 'n', 'n', 'n', 'i'],
 ['i', 'h', 'j', 'h', 'i'],
 ['e', 'f', 's', 'f', 'e'],
 ['e', 'f', 'v', 'f', 'e']]

Assim como determinado pelo enunciado, todos os palíndromos gerados possuem pelo menos uma vogal e são compostos por cinco letras!

## ☺️ **Conclusão** 

A partir do desenvolvimento do notebook, foi possível concluir que o algoritmo genético demonstrou alta performance, sendo capaz de gerar 10 palíndromos válidos (com 5 letras e ao menos uma vogal) em menos de 20 gerações na maioria dos casos. Isso evidencia a eficácia das estratégias escolhidas de seleção, cruzamento e mutação adotadas na resolução do problema, bem como boa estruturação da função objetivo e da função responsável por criar letras repetidas, feitas de forma autoral. 

## 🗃️ **Referências!** 

[1] Palíndromos: o que são e mais de 100 exemplos. Dicio, Dicionário Online de Português. Disponível em: <https://www.dicio.com.br/lista-palindromos/>. Acesso em: 6 jun. 2025.